In [ ]:
import psycopg2
import sys
import pandas as pd

In [ ]:
conn_string = "host='localhost' dbname='test'"

# print the connection string we will use to connect
print("Connecting to database\n	{0}".format(conn_string))

# get a connection, if a connect cannot be made an exception will be raised here
conn = psycopg2.connect(conn_string)

# conn.cursor will return a cursor object, you can use this cursor to perform queries
cursor = conn.cursor()
print("Connected!\n")

In [ ]:
# Reads data from file
data = pd.read_csv('test.csv')
data.head()

In [ ]:
# FUNCTION TO CREATE TABLE IN POSTGRES BASED ON PANDAS DF.
# MISSING PRIMARY KEY (PROBABLY FIRST VAR BY DEFAULT?)
def create_table(table_name, data):
    query = "CREATE TABLE IF NOT EXISTS {} (".format(table_name)
    for i, col in enumerate(data.columns):
        col_type=data.dtypes[i]
        # MORE TYPES, RAISE ERROR
        if col_type == 'O':
            postgres_type = 'VARCHAR'
        elif col_type == 'int64':
            postgres_type = 'INT'
        string_var = "{} {} NOT NULL".format(col, postgres_type)
        
        if i==0:# Very ugly code (well, all the function is ugly but it works)
            query = query + string_var
        else:
            query = query + ", " + string_var
    query = query + ")"
    
    cursor.execute(query)
    conn.commit()

In [ ]:
def upload_data(table_name, file_path):
    f = open(file_path, 'r')
    next(f)
    cursor.copy_from(f, table_name, sep=',')
    f.close()
    conn.commit()

In [ ]:
create_table('census_test', data)

In [ ]:
upload_data('census_test', 'test.csv')

In [ ]:
# CHECK
cursor.execute("SELECT * from census_test;")
cursor.fetchall()